In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 1.9 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00


In [ ]:
import torch
import accelerate
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")
model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

In [ ]:
def read_data(input_file):
    with open(input_file, "r", encoding="utf-8") as f:
        columns = f.readline().strip().split('\t')
        data = {col_name: [] for col_name in columns}
        num_cols = len(columns)
        n = 0
        for line in f:
            n += 1
            line = line.replace('\n', "")
            values = line.split("\t")
            if len(values) != num_cols:
                print(line)
                print(values)
                raise ValueError(f"Expected {num_cols} columns, found {len(values)} columns!")
            for i in range(len(values)):
                data[columns[i]].append(values[i])

    # ['id', 'title', 'context', 'question', 'has_answer', 'start_index', 'answer']

    del data["title"]
    data["size"] = n
    return data

In [ ]:
def convert_data_to_samples(data, tokenizer, max_chunk_size=512, chunk_overlap_size=128):
  def find_context_start_index(sequence_ids):
    for i in range(len(sequence_ids)):
      if sequence_ids[i] == tokenizer.eos_token_id:
        return i + 1
    return 1

  def find_context_end_index(sequence_ids):
    for i in range(len(sequence_ids)-1, 0, -1):
      if sequence_ids[i] == tokenizer.eos_token_id:
        return i - 1
    return len(sequence_ids) - 2

  questions = data["question"]
  contexts = data["context"]
  has_answers = data["has_answer"]
  start_indices = data["start_index"]
  answers = data["answer"]

  tokenized_data = {'labels': [], 'input_ids': [], 'attention_mask': []}

  for i in tqdm(range(data["size"])):
    question = questions[i]
    context = contexts[i]
    has_answer = int(has_answers[i])
    start_index = int(start_indices[i])
    answer = answers[i]

    answer_len = len(answer)

    end_index = start_index + answer_len

    encoding = tokenizer(question,
                         context,
                         truncation="only_second",
                         max_length=max_chunk_size,
                         stride=chunk_overlap_size,
                         return_overflowing_tokens=True,
                         return_offsets_mapping=True,
                         padding="max_length")

    for input_ids, attention_mask, offset_mapping in zip(encoding.input_ids, encoding.attention_mask, encoding.offset_mapping):

      context_start_index = find_context_start_index(input_ids)
      context_end_index = find_context_end_index(input_ids)

      # print("start index:", context_start_index)
      # print("end index:", context_end_index)
      # print(len(offset_mapping))

      if bool(has_answer):
        if offset_mapping[context_start_index][0] <= start_index and offset_mapping[context_end_index][1] >= end_index:
          label = "Có. Câu trả lời: " + answer
        else:
          label = "Không có câu trả lời"
      else:
        label = "Không có câu trả lời"

      label = tokenizer(label, max_length=chunk_overlap_size, truncation=True, padding=True)

      tokenized_data["input_ids"].append(input_ids)
      tokenized_data["attention_mask"].append(attention_mask)
      tokenized_data["labels"].append(label.input_ids)

  return tokenized_data


In [ ]:
# !gdown --id 1JwUYWOaYU71vesQHLsmRxxeSQKMLV14F
# !gdown --id 1MX2nS5uTns7aUfAn71kJWt-2pmBcKVQh
!gdown -O dev.tsv 11EbwMMDcsMVWvvfi7JqaliMe9Y14EuOc
!gdown -O train.tsv 1O2ogWFZ-4ddL5n-uUufXz2cM6qkJwUPq

Downloading...
From: https://drive.google.com/uc?id=11EbwMMDcsMVWvvfi7JqaliMe9Y14EuOc
To: /content/dev.tsv
100% 18.7M/18.7M [00:00<00:00, 188MB/s]
Downloading...
From: https://drive.google.com/uc?id=1O2ogWFZ-4ddL5n-uUufXz2cM6qkJwUPq
To: /content/train.tsv
100% 186M/186M [00:03<00:00, 54.6MB/s]


In [ ]:
def create_dataset(input_file, tokenizer):
  data = read_data(input_file)
  dict_obj = convert_data_to_samples(data, tokenizer)
  ds = Dataset.from_dict(dict_obj)
  return ds

In [ ]:
train_dataset = create_dataset("train.tsv", tokenizer)
val_dataset = create_dataset("dev.tsv", tokenizer)

  0%|          | 0/160843 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")


training_args = Seq2SeqTrainingArguments("tmp/",
                                      do_train=True,
                                      do_eval=True,
                                      num_train_epochs=10,
                                      learning_rate=2e-5,
                                      warmup_ratio=0.05,
                                      weight_decay=0.001,
                                      per_device_train_batch_size=4,
                                      per_device_eval_batch_size=4,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="epoch",
                                      save_total_limit=3,
                                      eval_steps=1000,
                                      #evaluation_strategy="steps",
                                      # evaluation_strategy="no",
                                      fp16=True,
                                      )

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

trainer.train()

In [ ]:
!zip -r states.zip /content/tmp/checkpoint-21570/

In [ ]:
!du -sh states.zip

2.4G	states.zip


In [ ]:
!cp -r states.zip /content/drive/MyDrive/save/

In [ ]:
sample_encoding = tokenizer("Thành phần chính của phở là gì",
                            "Nó cũng là một thành phần được sử dụng trong nấu nước dùng cho món phở của người Việt Nam .",
                            max_length=512,
                            padding="max_length", return_tensors="pt")

In [ ]:
max_target_length = 256
outputs = model.generate(
      input_ids=sample_encoding['input_ids'].to(device),
      max_length=max_target_length,
      attention_mask=sample_encoding['attention_mask'].to(device),
  )
with tokenizer.as_target_tokenizer():
    outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
outputs

['Có. Câu trả lời: thịt tươi']

In [ ]:
max_target_length = 256
dataloader = torch.utils.data.DataLoader(test_tokenized_datasets, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(tqdm(dataloader)):
  outputs = model.generate(
      input_ids=batch['input_ids'].to('cuda'),
      max_length=max_target_length,
      attention_mask=batch['attention_mask'].to('cuda'),
  )
  with tokenizer.as_target_tokenizer():
    outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

    labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
    actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
  predictions.extend(outputs)
  references.extend(actuals)

# Finetune

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!gdown 1M90GZ0KxlbG6v1tNOWx913z38sDy_GSL

Downloading...
From: https://drive.google.com/uc?id=1M90GZ0KxlbG6v1tNOWx913z38sDy_GSL
To: /content/train.csv
100% 7.30M/7.30M [00:00<00:00, 27.4MB/s]


In [ ]:
!gdown 1FSpKNQnAFoE_iO8yFq9mq-aIC7nG2c7C

Downloading...
From: https://drive.google.com/uc?id=1FSpKNQnAFoE_iO8yFq9mq-aIC7nG2c7C
To: /content/val.csv
100% 464k/464k [00:00<00:00, 148MB/s]


In [ ]:
def read_zqa_data(input_file):
    data = {"question": [], "context": [], "label": [] }
    with open(input_file, "r", encoding="utf-8") as f:
        n = 0
        for line in f:
            question, context, answer = line.strip().split('\t')
            n += 1
            data["question"].append(question)
            data["context"].append(context)
            data["label"].append(True if answer.strip() == "true" else False)

    data["size"] = n
    return data

In [ ]:
def convert_zqa_data_to_samples(data, tokenizer, max_length=512):
  questions = data["question"]
  contexts = data["context"]
  labels = data["label"]

  tokenized_data = {'labels': [], 'input_ids': [], 'attention_mask': []}

  for i in tqdm(range(data["size"])):
    question = questions[i]
    context = contexts[i]
    label = labels[i]

    encoding = tokenizer(question,
                         context,
                         truncation=True,
                         max_length=max_length,
                         padding="max_length")

    input_ids = encoding.input_ids
    attention_mask = encoding.attention_mask
    if label:
      label = "Có câu trả lời"
    else:
      label = "Không có câu trả lời"

    label = tokenizer(label, max_length=8, truncation=True, padding="max_length")

    tokenized_data["input_ids"].append(input_ids)
    tokenized_data["attention_mask"].append(attention_mask)
    tokenized_data["labels"].append(label.input_ids)

  return tokenized_data


In [ ]:
def create_zqa_dataset(input_file, tokenizer):
  data = read_zqa_data(input_file)
  dict_obj = convert_zqa_data_to_samples(data, tokenizer)
  ds = Dataset.from_dict(dict_obj)
  return ds

In [ ]:
train_zqa_dataset = create_zqa_dataset("train.csv", tokenizer)

  0%|          | 0/16994 [00:00<?, ?it/s]

In [ ]:
val_zqa_dataset = create_zqa_dataset("val.csv", tokenizer)

  0%|          | 0/1114 [00:00<?, ?it/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")


training_args = Seq2SeqTrainingArguments("tmp_zqa/",
                                      do_train=True,
                                      do_eval=True,
                                      num_train_epochs=10,
                                      learning_rate=2e-5,
                                      warmup_ratio=0.05,
                                      weight_decay=0.001,
                                      per_device_train_batch_size=4,
                                      per_device_eval_batch_size=4,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="epoch",
                                      save_total_limit=3,
                                      #eval_steps=1,
                                      #evaluation_strategy="steps",
                                      # evaluation_strategy="no",
                                      fp16=True,
                                      )

In [ ]:
zqa_trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_zqa_dataset,
    eval_dataset=val_zqa_dataset,
    data_collator=data_collator,
)

zqa_trainer.train()

In [ ]:
!zip -r finetune-states.zip /content/tmp_zqa/checkpoint-12747

  adding: content/tmp_zqa/checkpoint-12747/ (stored 0%)
  adding: content/tmp_zqa/checkpoint-12747/training_args.bin (deflated 50%)
  adding: content/tmp_zqa/checkpoint-12747/config.json (deflated 47%)
  adding: content/tmp_zqa/checkpoint-12747/trainer_state.json (deflated 77%)
  adding: content/tmp_zqa/checkpoint-12747/generation_config.json (deflated 27%)
  adding: content/tmp_zqa/checkpoint-12747/model.safetensors (deflated 8%)
  adding: content/tmp_zqa/checkpoint-12747/optimizer.pt (deflated 11%)
  adding: content/tmp_zqa/checkpoint-12747/scheduler.pt (deflated 56%)
  adding: content/tmp_zqa/checkpoint-12747/rng_state.pth (deflated 25%)


In [ ]:
!cp -r finetune-states.zip /content/drive/MyDrive/save/

# Test

In [ ]:
import numpy as np

max_target_length = 8
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")
test_zqa_dataloader = torch.utils.data.DataLoader(val_zqa_dataset, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(tqdm(test_zqa_dataloader)):
  outputs = model.generate(
      input_ids=batch['input_ids'].to(device),
      max_length=max_target_length,
      attention_mask=batch['attention_mask'].to(device),
  )
  with tokenizer.as_target_tokenizer():
    outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

    labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
    actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
  predictions.extend(outputs)
  references.extend(actuals)



  0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
predictions
references

In [ ]:
references

['Không có câu trả lời',
 'Có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Có câu trả lời',
 'Có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Có câu

In [ ]:
predictions

['Không có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 'Không có câu trả lời',
 '

In [ ]:
def check_valid_prediction(output_predictions):
  s = set()
  for answer in output_predictions:
    s.add(answer.strip())

  return s

In [ ]:
num_distinct_answer = check_valid_prediction(predictions)

In [ ]:
num_distinct_answer

{'Có câu trả lời', 'Không có câu trả lời'}

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
bool_references = []
bool_predictions = []

for y_true, y_pred in tqdm(zip(references, predictions)):
  if y_true == "Có câu trả lời":
    bool_references.append(1)
  elif y_true == "Không có câu trả lời":
    bool_references.append(0)
  else:
    raise ValueError("Error")

  if y_pred == "Có câu trả lời":
    bool_predictions.append(1)
  elif y_pred == "Không có câu trả lời":
    bool_predictions.append(0)
  else:
    raise ValueError("Error")

0it [00:00, ?it/s]

In [ ]:
f1_score(bool_references, bool_predictions, average="weighted")

0.8712556361114986


In [ ]:
f1_score(bool_references, bool_predictions, average="macro")

0.8424137596776428
